* https://arxiv.org/pdf/1505.04597.pdf -- U-net (2015)
* https://arxiv.org/pdf/1706.05587.pdf -- DeepLabV3+ (2017)
    * https://arxiv.org/pdf/1802.02611v3.pdf -- (2018)
    * https://arxiv.org/pdf/1210.5644.pdf -- CRFs (2012)
* https://arxiv.org/pdf/1612.01105.pdf -- PSPNet (2017)
* https://arxiv.org/pdf/1902.09212.pdf -- HRNetV2 (2019)
* https://arxiv.org/pdf/1810.11579.pdf -- $A^2$ Nets (2018)